In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers
!pip install language_tool_python
!pip install pymorphy2
!pip install stanza
!pip install spacy
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=75b23347b48be09b0c83c680d2f66a7538e08dd46b114b6e2458eeb2c4184747
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Su

In [2]:
import pymorphy2
import itertools
morph = pymorphy2.MorphAnalyzer()

In [3]:
cases = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
numbers = ['sing', 'plur']
genders = ['masc', 'femn', 'neut']
tenses = ['past', 'pres', 'futr']

In [4]:
def get_word(p, param):
  if len(param) == 3:
    word = p.inflect({param[0], param[1], param[2]})
  else:
    word = p.inflect({param[0], param[1]})
  if word is not None:
    return word.word
  else:
    return None

def get_inflected_words(keyword):
  p = morph.parse(keyword)[0]
  inflected_words = []
  if 'VERB' in p.tag or 'INFN' in p.tag:
    for param in itertools.product(numbers, tenses, genders):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
    for param in itertools.product(numbers, tenses):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
    for param in itertools.product(tenses, genders):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
  else:
    for param in itertools.product(numbers, cases, genders):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
    for param in itertools.product(numbers, cases):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
    for param in itertools.product(cases, genders):
      word = get_word(p, param)
      if word is not None:
        inflected_words.append(word)
  return list(set(inflected_words))

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from silero import silero_te
import language_tool_python
import pymorphy2
import string
import torch
import copy
import json

import torch.nn.functional as F
from nltk.stem.snowball import SnowballStemmer

from text_analysis.segmentation import get_sentences
from text_analysis.tokenization import get_tokens_natasha
from text_analysis.morphology import get_morph_pymorphy
import text_analysis

# ---------------- SETTINGS SECTION ----------------

# Number of attempts for correcting ending of the word
attempts_change_ending = 5

proba_gce_treshold = 0.01

# list of languages to load
languages = ['ru']  # ['ru', 'en', 'es', 'de', 'fr', 'ar', 'tr', 'uk']

# --------------- LOAD MODELS ----------------

ner_models = {}

porter_stem = None
silero_model = None

def init():
    global languages, porter_stem, silero_model, tokenizer_bert, model_bert,\
        PRE_TRAINED_MODEL_NAME_BERT

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    if 'ru' in languages:
        PRE_TRAINED_MODEL_NAME_BERT = "ai-forever/rugpt3large_based_on_gpt2"
        tokenizer_bert = GPT2Tokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
        model_bert = GPT2LMHeadModel.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
        model_bert.eval()
        model_bert.to(device)
        porter_stem = SnowballStemmer("russian")
        silero_model, example_texts, languages, punct, apply_te = silero_te()
# --------------------------------------------------

# ------------- WORD ENDING CORRECTION -------------

def correct_stem(stem, tokenizer_bert):
    bert_predicted_index = tokenizer_bert.encode(stem)[1:-1]
    roberta_tokens = []
    for index in bert_predicted_index:
        roberta_tokens.append(tokenizer_bert.decode([index]))
    if len(roberta_tokens[-1]) == 1:
        stem = tokenizer_bert.decode(bert_predicted_index[:-1])
    return stem


def check_match_lemmas(keyword_parsed, predicted_keyword_parsed):
    if predicted_keyword_parsed['lemma'] == keyword_parsed['lemma'] \
            and (('sing' in predicted_keyword_parsed['feats'] and 'sing' in keyword_parsed['feats'])
                 or ('plur' in predicted_keyword_parsed['feats'] and 'plur' in keyword_parsed['feats'])):
        return True
    else:
        return False


def get_corrected_keyword(keyword, keyword_stem, predicted_token, lang):
    punct = string.punctuation + '...'
    if predicted_token not in punct:
        grammar = get_tokens_natasha(keyword)
        grammar = get_morph_pymorphy(grammar, '', True)
        keyword_parsed = {'lemma': grammar[0][0], 'feats': grammar[0][3]}

        predicted_keyword = keyword_stem + predicted_token
        grammar = get_tokens_natasha(predicted_keyword)
        grammar = get_morph_pymorphy(grammar, '', True)
        predicted_keyword_parsed = {'lemma': grammar[0][0], 'feats': grammar[0][3]}

        if check_match_lemmas(keyword_parsed, predicted_keyword_parsed):
            return predicted_keyword

        predicted_keyword = keyword_stem[:-1] + predicted_token
        grammar = get_tokens_natasha(predicted_keyword)
        grammar = get_morph_pymorphy(grammar, '', True)
        predicted_keyword_parsed = {'lemma': grammar[0][0], 'feats': grammar[0][3]}
        if check_match_lemmas(keyword_parsed, predicted_keyword_parsed):
            return predicted_keyword

        predicted_keyword = porter_stem.stem(keyword) + predicted_token
        grammar = get_tokens_natasha(predicted_keyword)
        grammar = get_morph_pymorphy(grammar, '', True)
        predicted_keyword_parsed = {'lemma': grammar[0][0], 'feats': grammar[0][3]}
        if check_match_lemmas(keyword_parsed, predicted_keyword_parsed):
            return predicted_keyword

    return ''


def correct_keywords_col_bert(text, keywords_col, tokenizer_bert, model_bert, proba_gce_treshold, model_name, lang):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    sentences = [sentence['text'] for sentence in get_sentences(text, lang, 0, 0)]
    predicted_keywords = []
    print('\t\tSentences:')
    for sentence in sentences:
        print('\t\t\t' + sentence)
    index_sentence = 0
    index_keyword_col = 0
    start_index_search_kw = 0
    while index_sentence < len(sentences) and index_keyword_col < len(keywords_col):
        keyword_col = keywords_col[index_keyword_col]
        sentence = sentences[index_sentence]
        if keyword_col in str(sentence[start_index_search_kw:]):
            print('\n\t\tSentence: ' + sentence)
            start_index_keyword_col = sentence.index(keyword_col, start_index_search_kw)
            prefix = sentence[:start_index_keyword_col].strip()
            tail = sentence[start_index_keyword_col + len(keyword_col):]
            start_index_search_kw += len(keyword_col)

            grammar = get_tokens_natasha(keyword_col)
            grammar = get_morph_pymorphy(grammar, '', False)
            keyword_col_token_pos = [[item[0], item[2]] for item in grammar]

            first_noun_index = 0
            while first_noun_index < len(keyword_col_token_pos) - 1 and keyword_col_token_pos[first_noun_index][1] != 'NOUN':
                first_noun_index += 1

            for j in reversed(range(first_noun_index + 1, len(keyword_col_token_pos))):
                tail = ' ' + keyword_col_token_pos[j][0] + tail
            for i in reversed(range(first_noun_index + 1)):
                keyword = keyword_col_token_pos[i][0]
                keyword_stem_nltk = porter_stem.stem(keyword)
                #keyword_stem = porter_stem.stem(keyword)
                bert_predicted_indexes = tokenizer_bert.encode(keyword)
                token_list = []
                for index in bert_predicted_indexes:
                  token_list.append(tokenizer_bert.decode(index))
                if len(token_list) > 1:
                  keyword_stem = ''.join(token_list[:-1])
                elif len(token_list) == 1:
                  keyword_stem = ''.join(token_list)
                else:
                  keyword_stem = None
                print(f'Список токенов: {token_list}')
                context = prefix + ''.join(
                    [' ' + x[0] for x in keyword_col_token_pos[:i]]) + ' ' + keyword_stem
                
                context_w_o_stem = prefix + ''.join(
                    [' ' + x[0] for x in keyword_col_token_pos[:i]]) + ' '
                print('\t\tContext: ' + context)

                num_return_sequences = 1000
                input_tokens = tokenizer_bert.encode(context_w_o_stem, return_tensors="pt").cuda()

                with torch.no_grad():
                    outputs = model_bert.generate(input_tokens.cuda(), num_return_sequences= num_return_sequences,
                                   max_length=10, 
                                   repetition_penalty=2.0,
                                   do_sample=True,
                                   top_k=50, top_p=0.95,
                                   temperature=0.7)
                '''
                print('Варианты: ')
                for i in range(num_return_sequences):
                  print(tokenizer_bert.decode(outputs[i]))
                '''
              

                print('\t\tKeyword stem: ' + keyword_stem)
                k = 0
                found = False
                final_predicted_keyword = ''
                inflected_words = get_inflected_words(keyword)
                print(f'Похожие слова: {inflected_words}')
                while k < num_return_sequences:
                    generated_text = tokenizer_bert.decode(outputs[k])
                    predicted_token = generated_text.replace(context_w_o_stem, '').split()[0]
                    if predicted_token.startswith('##'):
                        predicted_token = predicted_token.replace('##', '')
                    else:
                        if not 'roberta' in model_name.lower():
                            predicted_token = ' ' + predicted_token
                    punct = string.punctuation + '...'
                    if keyword_stem == '':
                      predicted_keyword = keyword_stem + predicted_token
                    else:
                      predicted_keyword = predicted_token
                    predicted_keywords.append(predicted_keyword)
                    print('\t\t\t' + str(k + 1) + ' predicted token: ' + predicted_token)
                    if len(predicted_keyword) > 0 and not found:
                      if predicted_keyword in inflected_words or keyword_stem in predicted_keyword or keyword_stem_nltk in predicted_keyword:
                        final_predicted_keyword = predicted_keyword
                        found = True
                        break
                    k += 1
                if not found and keyword not in tail:
                    tail = ' ' + keyword + tail
                elif found:
                    tail = ' ' + final_predicted_keyword + tail

            print('\t\tCorrected sentence: ' + prefix + tail)
            sentences[index_sentence] = prefix + tail
            sentences[index_sentence] = sentences[index_sentence].strip()
            index_keyword_col += 1
        else:
            start_index_search_kw = 0
            index_sentence += 1
    return ' '.join(sentence for sentence in sentences), predicted_keywords, token_list

# --------------------------------------------------

#исправление грамматических и орфографических ошибок с помощью LanguageTool (локально)
def grammar_correction(text, lang, replaces_dict=None):
    if replaces_dict != None:
        replaces = [elem['new_string'] for elem in replaces_dict if elem['check']]
        replaces_indexes = [[elem['result_start'], elem['result_start'] + len(elem['new_string']) -1] for elem in replaces_dict if elem['check']]
    if not lang:
        tool = language_tool_python.LanguageTool('auto')
    else:
        tool = language_tool_python.LanguageTool(lang)
    if replaces_dict == None:
        errors = tool.check(text)
        matches = []
        for error in errors:
            if error.replacements is not None and error.offset is not None:
                error_dict = {'error_word': text[error.offset:error.offset + error.errorLength],
                            'start': error.offset, 'end': error.offset + error.errorLength - 1,
                            'replacements': error.replacements}
                matches.append(error_dict)
        text_corrected = tool.correct(text)
        grammar_dict = {'corrected_text': text_corrected, 'number_errors': len(errors)}
        if len(errors) > 0:
            grammar_dict['errors'] = matches
    else:
        number_errors = 0
        text_corrected = text
        matches = []
        for i, word in enumerate(replaces):
            if word == text[replaces_indexes[i][0]:replaces_indexes[i][1]]:
                errors = tool.check(word)
                number_errors += len(errors)
                for error in errors:
                    if error.replacements is not None and error.offset is not None:
                        error_dict = {'error_word': text[error.offset:error.offset + error.errorLength],
                                    'start': error.offset, 'end': error.offset + error.errorLength - 1,
                                    'replacements': error.replacements}
                        matches.append(error_dict)
                word_corrected = tool.correct(word)
                text_corrected[replaces_indexes[i][0]:replaces_indexes[i][1]] == word_corrected
        text_corrected = correct_sentence(text_corrected)
        grammar_dict = {'corrected_text': text_corrected, 'number_errors': number_errors}
        if number_errors > 0:
            grammar_dict['errors'] = matches
    return grammar_dict


#исправление пунктуционных ошибок в сегментированном тексте на предложения (предварительно очистив его от всех знаков пунктуации)
def punctuation(text, lang, replaces_dict=None):
    if replaces_dict != None:
        replaces = [elem['new_string'] for elem in replaces_dict if elem['check']]
        replaces_indexes = [[elem['result_start'], elem['result_start'] + len(elem['new_string']) -1] for elem in replaces_dict if elem['check']]
    #функция, расставляющая знаки препинания с помощью моделей Silero (silero_te)
    if replaces_dict == None:
        sentences = get_sentences_without_punctuation(text, lang)
        new_sentences = []
        for sent in sentences:
            sent = sent.lower()
            output_text = silero_model.enhance_text(sent, lang)
            new_sentences.append(output_text)
        return ' '.join(new_sentences)
    else:
        output_text = text
        for i, word in enumerate(replaces):
            if word == text[replaces_indexes[i][0]:replaces_indexes[i][1]]:
                output_word = silero_model.enhance_text(word, lang)
                output_text[replaces_indexes[i][0]:replaces_indexes[i][1]] = output_word
        return output_text
   

#удаление пробелов перед знаками препинания
def correct_sentence(sentence):
    for punct in string.punctuation:
        if punct in sentence:
            sentence = sentence.replace(' ' + punct, punct)
    return sentence


#очистка предложений от пунктуации
def get_sentences_without_punctuation(text, lang):
    sentences = []
    punct = string.punctuation
    for sent in get_sentences(text, lang, min_sent_len=0, min_sent_words=0):
        sentence = sent['text'].translate(str.maketrans('', '', punct))
        sentences.append(sentence)
    return sentences


def agreement(text, keywords, lang):
    corrected_sentence, predicted_keywords = correct_keywords_col_bert(text, keywords, tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang)
    grammar_dict = {'corrected_text': corrected_sentence, 'replacements': predicted_keywords}
    return grammar_dict


def dep_word_agreement(original_text, changed_text, replaces, lang):
    #токенизация исходного и конечного предложений
    original_tokens = [item[0] for item in get_tokens_nltk(original_text, lang)]
    changed_tokens = [item[0] for item in get_tokens_nltk(changed_text, lang)]
    #получение списка изменённых слов в changed_text и их индексов
    unique_changed_words = []
    replaced_words = []
    replaced_words_indexes = []
    for elem in replaces:
        if elem['check']:
            replaced_words.append(elem['new_string'])
            replaced_words_indexes.append([elem['result_start'], elem['result_start'] + len(elem['new_string']) - 1])
    changed_words_indexes = []
    for i in range(len(changed_tokens)):
        for j in range(len(replaced_words)):
            if changed_tokens[i] == replaced_words[j] and changed_tokens[i] == changed_text[replaced_words_indexes[j][0]: replaced_words_indexes[j][1] + 1]:
                unique_changed_words.append(changed_tokens[i])
                changed_words_indexes.append(i)
    #получение grammar на основе токенов исходного предложения для использования внешней функции без изменений
    original_tokens_grammar = [[token, '', '', '', '', ''] for token in original_tokens]
    changed_tokens_grammar = [[token, '', '', '', '', ''] for token in changed_tokens]
    #добавление в grammar значений параметров word_id и head_id из внешней функции синтаксического анализа
    original_syntax = get_syntax_stanza([original_tokens_grammar], lang)
    changed_syntax = get_syntax_stanza([changed_tokens_grammar], lang)
    #получение списка head_ids (списка индексов связанных слов)
    head_ids = [item[5] for item in changed_tokens_grammar]
    #получение списка связанных слов и списка их индексов в changed_text
    linked_words_list = []
    linked_words_indexes = []
    for index in changed_words_indexes:
        linked_words = []
        for i in range(len(head_ids)):
            if index + 1 in head_ids:
                if index + 1 == head_ids[i]:
                    linked_words.append(changed_tokens[i])
                    linked_words_indexes.append(i)
        linked_words_list.append(linked_words)
    #реализация морфологии для слов из unique_changed_words и связанных слов из linked_words
    morph = pymorphy2.MorphAnalyzer()
    morph_unique_list = []
    morph_linked_list = []
    for word in unique_changed_words:
        morph_unique_list.append(morph.parse(word)[0])
    for i in range(len(linked_words_list)):
        changed_words_list = []
        for elem in linked_words_list[i]:
            changed_words_list.append(morph.parse(elem)[0])
        morph_linked_list.append(changed_words_list)
    #получение списка морфологических признаков для слов из unique_changed_words
    morph_tags = [item.tag for item in morph_unique_list]
    #получение списка связанных слов с перенесёнными морфологическими признаками относительно признаков слов из unique_changed_words
    inflected_words_list = []
    for i, tag in enumerate(morph_tags):
        inflected_words = []
        for elem in morph_linked_list[i]:
            if 'VERB' in tag or 'GRND' in tag:
                inflected_words.append(elem.inflect({tag.number, tag.gender}).word)
            elif 'ADJF' in tag or 'ADJS' in tag or 'PRTF' in tag or 'PRTS' in tag:
                inflected_words.append(elem.inflect({tag.number, tag.gender, tag.case}).word)
        inflected_words_list.append(inflected_words)
    #замена форм связанных слов в конечном предложении
    for elem in inflected_words_list:
        if len(elem) > 0:
            linked_word_index = 0
            for i in range(len(changed_tokens)):
                if i in linked_words_indexes:
                    changed_tokens[i] = elem[linked_word_index]
                    linked_word_index += 1
    #возврат правильного предложения
    grammar_dict = {'corrected_text': correct_sentence(' '.join(changed_tokens))}
    return grammar_dict

# Parameter method can take one of the values "spelling", "punctuation", "grammar"
def error_correction(messages, methods=["spelling", 'punctuation', 'grammar']):
    messages_new = copy.deepcopy(messages)
    for message in messages_new:
        text = message['text']
        replaces = None
        if 'replaces' in message.keys():
            replaces = message["replaces"]
        lang = None
        if 'lang' in message:
            lang = message['lang']
        grammar_dict = {}
        for method in methods:
            if method == 'spelling':
                if replaces == None:
                    grammar_dict[method] = grammar_correction(text, lang)
                else:
                    grammar_dict[method] = grammar_correction(message['result_text'], lang, replaces)
            elif method == 'punctuation':
                if lang is not None:
                    if replaces == None:
                        grammar_dict[method] = {'corrected_text': punctuation(text, lang)}
                    else:
                        grammar_dict[method] = {'corrected_text': punctuation(message['result_text'], lang, replaces)}
                else:
                    grammar_dict[method] = {'corrected_text': text}
            elif method == 'grammar':
                if 'replaced_words' in message:
                    grammar_dict[method] = agreement(text, message['replaced_words'], lang)
                if 'result_text' in message:
                    grammar_dict[method] = dep_word_agreement(text, message['result_text'], replaces, lang)
        if len(grammar_dict) > 0:
            message['error_correction'] = grammar_dict
    return messages_new

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import text_analysis
text_analysis.morphology.init()
text_analysis.segmentation.init()
text_analysis.tokenization.init()

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


In [9]:
examples = {
    'AGR_CASE': ['Город Каталония - после Ис Гр Войны - был таким грустном местом где ничего не хватало и опасно было ходить из за нужды в ремонте, но люди не были добиты а наоборот, они были полными надеждой.',
                 'Анархизм - политическая система, которая предполагает отсутствие государства и наличия стихийной народной власти.'],
    'AGR_NUM': ['Главное различия в том, что здесь я работаю в благотворительном обществе, а в Америке я больше всего работала в бизнесе или в университете.', 
                'Догма - это основные положение истины, неизменную любых обстоятельствах.'],
    'AGR_PERS': ['На премьеру фильма пришли не много людей.', 
                 'Другие факторы приводя к потере языка, такие как война, стихийные бедствия, голод, болезнь и тому подобные, которые часто вызывают гибель целых общин, и, конечно, мы не можем предвидеть и предотвратить их.'],
    'BREV': ['Например: современный культурный человек должен владеть каким-либо иностранным языком, быть способным работать с компьютером, в то время как в 19 веке культурный человек должен был умело владеть шпагой, или работать на земле, то есть культура всегда исторична и социальная.',
             'Торговля услугами и товарами примерно одинаковая в развитых странах и развивающихся странах.'],
    'GENDER': ['Почему Сталин одобрял монументальную архитектуру?&gt;. Я думаю, что Сталин одобрял такой стиль архитектуры, потому что это отображала рост и поднятие страны ее развитие).',
               'Translate the questions and answer them&gt;. Что было самый главной проблемой Петра Первого, когда он принимал престол?'],
    'NUM': ['Напишите черновик краткой аннотации к своей курсовой работе.&gt;. В работе рассматривается разные виды компьютерных игр и их влияние на образование детей.', 
            'bibliography&gt;. В статье автор анализирует отношение между языком и мышлением'],
}

In [10]:
keywords = [['грустном'], ['наличия']]
i = 0
for elem in examples['AGR_CASE']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 13, but `max_length` is set to 10. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


		Sentences:
			Город Каталония - после Ис Гр Войны - был таким грустном местом где ничего не хватало и опасно было ходить из за нужды в ремонте, но люди не были добиты а наоборот, они были полными надеждой.

		Sentence: Город Каталония - после Ис Гр Войны - был таким грустном местом где ничего не хватало и опасно было ходить из за нужды в ремонте, но люди не были добиты а наоборот, они были полными надеждой.
Список токенов: ['гру', 'стном']
		Context: Город Каталония - после Ис Гр Войны - был таким гру


OutOfMemoryError: ignored

In [10]:
keywords = [['различия'], ['основные']]
i = 0
for elem in examples['AGR_NUM']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			Главное различия в том, что здесь я работаю в благотворительном обществе, а в Америке я больше всего работала в бизнесе или в университете.

		Sentence: Главное различия в том, что здесь я работаю в благотворительном обществе, а в Америке я больше всего работала в бизнесе или в университете.
Список токенов: ['раз', 'ли', 'чия']
		Context: Главное разли


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: разли
Похожие слова: ['различиями', 'различиям', 'различий', 'различиях', 'различии', 'различия', 'различию', 'различием', 'различие']
			1 predicted token: Главное???
			2 predicted token: _неприятно.<s>
			3 predicted token: -
			4 predicted token: —
			5 predicted token: –
			6 predicted token: простить.<s>
			7 predicted token: потреблять
			8 predicted token: _непреодолимо.<s>
			9 predicted token: —
			10 predicted token: это,
			11 predicted token: Главное??
			12 predicted token: _то,
			13 predicted token: -
			14 predicted token: для
			15 predicted token: _для
			16 predicted token: _то,
			17 predicted token: _в
			18 predicted token: сделать
			19 predicted token: _это
			20 predicted token: ь,
			21 predicted token: это
			22 predicted token: это
			23 predicted token: истинного
			24 predicted token: —
			25 predicted token: бы
			26 predicted token: необходимо
			27 predicted token: того,
			28 predicted token: _это
			29 predicted token: –
			30 predict

In [11]:
keywords = [['пришли'], ['приводя']]
i = 0
for elem in examples['AGR_PERS']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			На премьеру фильма пришли не много людей.

		Sentence: На премьеру фильма пришли не много людей.
Список токенов: ['при', 'шли']
		Context: На премьеру фильма при


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: при
Похожие слова: ['прислала', 'прислал', 'пришлём', 'пришлю', 'прислало', 'прислали']
			1 predicted token: Дам
			2 predicted token: "Свой
			3 predicted token: "Призрак
			4 predicted token: "Брат
			5 predicted token: "Ночь
			6 predicted token: «Дорога»
			7 predicted token: "Сны
			8 predicted token: `Откуда
			9 predicted token: «Небеса
			10 predicted token: "Легенда
			11 predicted token: "Четыре
			12 predicted token: "Стиляги-2".
			13 predicted token: Олег
			14 predicted token: "Классный
			15 predicted token: "Кролик
			16 predicted token: "Не
			17 predicted token: `Пятьдесят
			18 predicted token: "Невский
			19 predicted token: "Анатомия
			20 predicted token: "Матильда".
			21 predicted token: «Планета
			22 predicted token: "Пираты
			23 predicted token: `Пятница-13'"
			24 predicted token: "Холодное
			25 predicted token: `Побег'"
			26 predicted token: "Любовь
			27 predicted token: «В
			28 predicted token: "Сорок
			29 predicted token: "Духless
	

In [ ]:
keywords = [['социальная'], ['одинаковая']]
i = 0
for elem in examples['BREV']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 49, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			Например: современный культурный человек должен владеть каким-либо иностранным языком, быть способным работать с компьютером, в то время как в 19 веке культурный человек должен был умело владеть шпагой, или работать на земле, то есть культура всегда исторична и социальная.

		Sentence: Например: современный культурный человек должен владеть каким-либо иностранным языком, быть способным работать с компьютером, в то время как в 19 веке культурный человек должен был умело владеть шпагой, или работать на земле, то есть культура всегда исторична и социальная.
Список токенов: ['социаль', 'ная']
		Context: Например: современный культурный человек должен владеть каким-либо иностранным языком, быть способным работать с компьютером, в то время как в 19 веке культурный человек должен был умело владеть шпагой, или работать на земле, то есть культура всегда исторична и социаль
		Keyword stem: социаль
			1 predicted token: социальна
		Corrected sentence: Например: современный культур

In [ ]:
keywords = [['отображала'], ['самый']]
i = 0
for elem in examples['GENDER']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			Почему Сталин одобрял монументальную архитектуру?&gt;.
			Я думаю, что Сталин одобрял такой стиль архитектуры, потому что это отображала рост и поднятие страны ее развитие).

		Sentence: Я думаю, что Сталин одобрял такой стиль архитектуры, потому что это отображала рост и поднятие страны ее развитие).
Список токенов: ['от', 'ображ', 'ала']
		Context: Я думаю, что Сталин одобрял такой стиль архитектуры, потому что это отображ
		Keyword stem: отображ
			1 predicted token: отображало
		Corrected sentence: Я думаю, что Сталин одобрял такой стиль архитектуры, потому что это отображало рост и поднятие страны ее развитие).
		Sentences:
			Translate the questions and answer them&gt;.
			Что было самый главной проблемой Петра Первого, когда он принимал престол?

		Sentence: Что было самый главной проблемой Петра Первого, когда он принимал престол?
Список токенов: ['сам', 'ый']
		Context: Что было сам
		Keyword stem: сам
			1 predicted token: сам
		Corrected sentence: Что было са

In [ ]:
keywords = [['рассматривается'], ['отношение']]
i = 0
for elem in examples['NUM']:
      corrected_sentence, predicted_keywords, token_list = correct_keywords_col_bert(elem, keywords[i], tokenizer_bert, model_bert,
                                                                        proba_gce_treshold, PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
      i += 1

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			Напишите черновик краткой аннотации к своей курсовой работе.&gt;.
			В работе рассматривается разные виды компьютерных игр и их влияние на образование детей.

		Sentence: В работе рассматривается разные виды компьютерных игр и их влияние на образование детей.
Список токенов: ['расс', 'матри', 'вается']
		Context: В работе рассматри


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: рассматри
			1 predicted token: рассматриваемого
		Corrected sentence: В работе рассматриваемого разные виды компьютерных игр и их влияние на образование детей.
		Sentences:
			bibliography&gt;.
			В статье автор анализирует отношение между языком и мышлением

		Sentence: В статье автор анализирует отношение между языком и мышлением
Список токенов: ['отно', 'шение']
		Context: В статье автор анализирует отно
		Keyword stem: отно
			1 predicted token: отношенческие
		Corrected sentence: В статье автор анализирует отношенческие между языком и мышлением


In [ ]:
import pandas as pd
def process_file(filenames):
    files_encoding = 'UTF-8'
    output_messages  = []
    if not os.path.exists(filenames[1]):
        with open(filenames[0], 'r', encoding=files_encoding) as f:
            input_messages = pd.read_excel(filenames[0])
        for i in range(len(input_messages)):
          sentence, keywords = correct_keywords_col_bert(input_messages.loc[i, 'Prepared Sentence'], [input_messages.loc[i, 'Error']], tokenizer_bert, model_bert, proba_gce_treshold=0.01, model_name=PRE_TRAINED_MODEL_NAME_BERT, lang='ru')
          output_messages.append({
              'Original Sentence': input_messages.loc[i, 'Original Sentence'],
              'Corrected Sentence': input_messages.loc[i,'Corrected Sentence'],
              'Tag': input_messages.loc[i,'Tag'],
              'Error': input_messages.loc[i,'Error'],
              'Correction': input_messages.loc[i,'Correction'],
              'Output Sentence': sentence,
              'Output Keywords': keywords, 
          })
        with open(filenames[1], 'w', encoding=files_encoding) as f:
            writer = pd.ExcelWriter(filenames[1])
            output_df = pd.DataFrame(output_messages)
            output_df.to_excel(writer)
            writer.save()

In [ ]:
process_file(file_list[7])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Sentences:
			Текст для Дни открытых дверей.

		Sentence: Текст для Дни открытых дверей.
		Context: Текст для дни


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: дни
			1 predicted token: днища
		Corrected sentence: Текст для днища открытых дверей.
		Sentences:
			Как вы понимаете, что такое идентичность?&gt;. чья-то идентичность это ответ на вопрос кто это такое?». такое идентичность формируется из всех разных вашей жизни, характера, идеологии, талантов и т.

		Sentence: Как вы понимаете, что такое идентичность?&gt;. чья-то идентичность это ответ на вопрос кто это такое?». такое идентичность формируется из всех разных вашей жизни, характера, идеологии, талантов и т.
		Context: Как вы понимаете, что так


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: сам
			1 predicted token: сам
		Corrected sentence: Что было сам главной проблемой Петра Первого, когда он принимал престол?
		Sentences:
			Продается квартира на Кудринской площади, находящееся в одной из семи сталинских высоток.

		Sentence: Продается квартира на Кудринской площади, находящееся в одной из семи сталинских высоток.
		Context: Продается квартира на Кудринской площади, находя


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


		Keyword stem: находя
			1 predicted token: находящеся
		Corrected sentence: Продается квартира на Кудринской площади, находящеся в одной из семи сталинских высоток.
		Sentences:
			Как вы понимаете фразу: «Мы рабы слов»?&gt;.
			За: необходимость осознать точку зрения, котором формируется на основе источников, доверность системы.

		Sentence: За: необходимость осознать точку зрения, котором формируется на основе источников, доверность системы.
		Context: За: необходимость осознать точку зрения, котор
		Keyword stem: котор
			1 predicted token: котор...
		Corrected sentence: За: необходимость осознать точку зрения, котор... формируется на основе источников, доверность системы.


<ipython-input-6-32cbb704ffd8>:23: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
